In [1]:
import sys
sys.path.append("../src")

import requests

from tonpy import LiteClient, Cell, get_block_info, BlockId, BlockIdExt, Address, Emulator, begin_cell
from tonpy.utils.shard_account import shard_is_ancestor, shard_child
from tonpy.autogen.block import Account, Transaction, Block, BlockInfo, BlockExtra, MessageAny, AccountState

from tonpy.autogen.block import Transaction
from deepdiff import DeepDiff

# Ignore some logs from ton monorepo
from tonpy.libs.python_ton import globalSetVerbosity
globalSetVerbosity(9)

In [2]:
# Example hashes:
#   27 txs in 1 block: 184C0C0EF74E279E63B363FCC81168AEB4C3769E4991FAF6982A5E232EE88541
#   After merge: 7A074921055E014CE6B5E7C929B12D9789F5419E771F1BD98C379A57DA15064A
#   TickTock: F8EAC78DA7005B67B4A473CB77CF90ACD4A31B883E0E6C2806ADB4C0D2B3E616

In [3]:
tx_hash = "52307C3D0F75FBD336814E39A3B349EFC1F15CAF03C22A6AD00DF8407531E293"

In [4]:
def get_tx_lt(tx_hash):
    query = '''query{transactions(hash: "%s"){lt,address,workchain,shard}}''' % tx_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['transactions'][0]
    return int(data['lt']), int(data['workchain']), str(data['address']), str(data['shard'])

In [5]:
# You can not trust this data from dton.io, but IF it's correct it'll be prooved by `get_transactions` later
tx_lt, wc, address, shard = get_tx_lt(tx_hash)
block_lt = tx_lt - tx_lt % 1000000

tx_account = Address(f"{wc}:{address}")

In [6]:
# Get random LiteServer
lc = LiteClient.get_one()

# When you get transaction - you automatically proof validness by tx_hash
transactions = lc.get_transactions(tx_account, tx_lt, tx_hash, 16).transactions

# We need to get all TXs in this block
whitelist_txs = [transactions[0]]
get_state_from = None
cur_index = 0

if len(transactions) > 1:
    done = transactions[0].blkid != transactions[1].blkid
else:
    done = True
    
while not done:
    cur_index += 1
    if cur_index > len(transactions) - 1:
        # Get new 16txs to load
        transactions.extend(lc.get_transactions(tx_account, 
                                          whitelist_txs[-1].prev_trans_lt, 
                                          whitelist_txs[-1].prev_trans_hash, 16).transactions)
    if cur_index > len(transactions):
        break
    
    if whitelist_txs[-1].blkid == transactions[cur_index].blkid:
        whitelist_txs.append(transactions[cur_index])
    else:
        done = True
        

[ 4][t 1][1706169432.940674067][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 132
[ 4][t 1][1706169433.012689114][BufferedFd.h:216][!outconn]	Flush write: +456B[left:0B]
[ 4][t 1][1706169433.119681120][BufferedFd.h:207][!outconn]	Flush read: +1392B[total:1392B]
[ 4][t 1][1706169433.119722128][adnl-ext-connection.cpp:80][!outconn]	len=64
[ 4][t 1][1706169433.119740963][adnl-ext-connection.cpp:163][!outconn]	received packet of size 64
[ 4][t 1][1706169433.119767189][adnl-ext-connection.cpp:80][!outconn]	len=1320
[ 4][t 1][1706169433.119807005][adnl-ext-connection.cpp:163][!outconn]	received packet of size 1320


In [7]:
# If there >1 TX in block - we need to emulate all of them before our one
print("To emulate, before our TX: ", len(whitelist_txs) - 1)

whitelist_txs = list(reversed(whitelist_txs))

To emulate, before our TX:  1


In [8]:
# Get block with prev state
# WARNING: this block is not proofed
state_block = whitelist_txs[0].blkid
current_block = lc.get_block_header(state_block)
print("Current block: ", current_block.blk_id, "\n")

# We need to get master block, so we can download libs and config
if wc != -1:
    # tlb python autogen
    block = Block().cell_unpack(current_block.virt_blk_root)
    block_info = BlockInfo().cell_unpack(block.info, True)
    
    # get master block id from WC
    master = BlockIdExt(id_=BlockId(-1, 0x8000000000000000, block_info.master_ref.master.seq_no), 
          file_hash=int(block_info.master_ref.master.file_hash, 2), 
          root_hash=int(block_info.master_ref.master.root_hash, 2))
else:
    master = current_block.blk_id

print("Master block: ", master, "\n")


Current block:  (0,8000000000000000,41346239):D3A6AAA846A92BEFDF747075359D5E9D2E8623197845E6E31A42CDACFF39FBFE:8F1A6C6E0D593B87BFB3B2C80E81017ADB558F9AAB0A32B10191F0698603883E 

Master block:  (-1,8000000000000000,35513921):FDDCDF2854A286EA90149C1685BCED2348321964C3E46FD6724755241FB136E5:B06B55BAD35ACD0AC73E3452618839723BEEA3F7913C5DE5C4132AED5A781319 



[ 4][t 1][1706169433.154532194][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 136
[ 4][t 1][1706169433.154762030][BufferedFd.h:216][!outconn]	Flush write: +204B[left:0B]
[ 4][t 1][1706169433.220935106][BufferedFd.h:207][!outconn]	Flush read: +620B[total:620B]
[ 4][t 1][1706169433.220976353][adnl-ext-connection.cpp:80][!outconn]	len=616
[ 4][t 1][1706169433.221018314][adnl-ext-connection.cpp:163][!outconn]	received packet of size 616


In [9]:
# Need to get rand_seed for block and get prev block with account state
current_full_block = lc.get_block(current_block.blk_id)

# It's stored in blockExtra
block = Block().cell_unpack(current_full_block)
block_info = BlockInfo().cell_unpack(block.info, True)
block_extra = BlockExtra().cell_unpack(block.extra, False)

rand_seed = int(block_extra.rand_seed, 2)

[ 4][t 1][1706169434.039869070][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 132
[ 4][t 1][1706169434.040161133][BufferedFd.h:216][!outconn]	Flush write: +200B[left:0B]
[ 4][t 1][1706169434.128634930][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:1288B]
[ 4][t 1][1706169434.128674269][adnl-ext-connection.cpp:80][!outconn]	len=106704
[ 4][t 1][1706169434.129024029][BufferedFd.h:207][!outconn]	Flush read: +3864B[total:5148B]
[ 4][t 1][1706169434.130497217][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:6436B]
[ 4][t 1][1706169434.130860090][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:7724B]
[ 4][t 1][1706169434.133571148][BufferedFd.h:207][!outconn]	Flush read: +5152B[total:12KB]
[ 4][t 1][1706169434.190451145][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:13KB]
[ 4][t 1][1706169434.191261053][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:15KB]
[ 4][t 1][1706169434.192240238][BufferedFd.h:207][!outconn]	Flush read: +3864B[total:18KB]
[ 

In [10]:
# To get prev block with account state - need to calculate correct shard (with account) if block is after merge
# If not after merge - just get prev seqno
if block_info.after_merge:    
    left = block_info.prev_ref.prev1
    left_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, True), left.seq_no),
                           root_hash=int(left.root_hash,2), file_hash=int(left.file_hash, 2))
    
    right = block_info.prev_ref.prev2
    right_shard = BlockIdExt(BlockId(wc, shard_child(current_block.blk_id.id.shard, False), right.seq_no),
                           root_hash=int(right.root_hash,2), file_hash=int(right.file_hash, 2))
    
    if shard_is_ancestor(left_shard.id.shard, tx_account.shard_prefix(60)):
        state_block = left_shard
    else:
        state_block = right_shard
else:
    current = block_info.prev_ref.prev
    state_block = BlockIdExt(BlockId(wc, current_block.blk_id.id.shard, current.seq_no),
                           root_hash=int(current.root_hash,2), file_hash=int(current.file_hash, 2))

In [11]:
print("State block: ", state_block, "\n")

State block:  (0,8000000000000000,41346238):038FC872DFF0507576D6D7E45BBD9E8177846633C92E47395502066865F1E556:554F004339E29CA1A9328991434C909BB86307FB55EC746A4C8EBB45074925BE 



In [12]:
# Load account state from prev block
account_state = lc.get_account_state(f"{wc}:{address}", state_block)

if not account_state.root.is_null():
    # Convert answer to AccountShardState
    account_state = begin_cell() \
        .store_ref(account_state.root) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64).end_cell()
else:
     account_state = begin_cell() \
        .store_ref(begin_cell().store_uint(0, 1).end_cell()) \
        .store_uint(int(account_state.last_trans_hash, 16), 256) \
        .store_uint(account_state.last_trans_lt, 64) \
        .end_cell()

[ 4][t 1][1706169435.884475946][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 168
[ 4][t 1][1706169435.884666920][BufferedFd.h:216][!outconn]	Flush write: +236B[left:0B]
[ 4][t 1][1706169436.834761143][BufferedFd.h:207][!outconn]	Flush read: +2528B[total:2528B]
[ 4][t 1][1706169436.834804058][adnl-ext-connection.cpp:80][!outconn]	len=2524
[ 4][t 1][1706169436.834863186][adnl-ext-connection.cpp:163][!outconn]	received packet of size 2524


In [13]:
# Load config of prev key block of MC block where TX was
# WARNING: this key block have no proof
key_block, config = lc.get_config_all(master, from_not_trusted_keyblock=True)

[ 4][t 1][1706169436.963984966][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 136
[ 4][t 1][1706169436.964226246][BufferedFd.h:216][!outconn]	Flush write: +204B[left:0B]
[ 4][t 1][1706169437.043871164][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:2576B]
[ 4][t 1][1706169437.043909311][adnl-ext-connection.cpp:80][!outconn]	len=64848
[ 4][t 1][1706169437.044141054][BufferedFd.h:207][!outconn]	Flush read: +3864B[total:6436B]
[ 4][t 1][1706169437.053758144][BufferedFd.h:207][!outconn]	Flush read: +6440B[total:12KB]
[ 4][t 1][1706169437.129198074][BufferedFd.h:207][!outconn]	Flush read: +9016B[total:21KB]
[ 4][t 1][1706169437.129261255][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:22KB]
[ 4][t 1][1706169437.129892111][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:25KB]
[ 4][t 1][1706169437.202817202][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:26KB]
[ 4][t 1][1706169437.203994274][BufferedFd.h:207][!outconn]	Flush read: +7728B[total:33KB]
[ 4][

In [14]:
print("Latest key block: ", key_block)

Latest key block:  (-1,8000000000000000,35512458):A16D2D7846937F4252CB7810E4407B6D1C37F0D480C75129A2F7AE320CD8C69F:F91F326F7B33E2C383F360917DADB8128057244A548CB87FD9F22B3EB8BE267A


In [15]:
# for emulation you need to get 16 latest mc blocks + 1 key block
old_mc_blocks = []
cur_seqno = master.id.seqno

for i in range(16):
    cur_seqno -= 1
    old_mc_blocks.append(lc.lookup_block(BlockId(-1, 0x8000000000000000, cur_seqno)).blk_id)

[ 4][t 1][1706169437.984529972][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 72
[ 4][t 1][1706169437.984745979][BufferedFd.h:216][!outconn]	Flush write: +140B[left:0B]
[ 4][t 1][1706169438.055859089][BufferedFd.h:207][!outconn]	Flush read: +540B[total:540B]
[ 4][t 1][1706169438.055901051][adnl-ext-connection.cpp:80][!outconn]	len=536
[ 4][t 1][1706169438.055937052][adnl-ext-connection.cpp:163][!outconn]	received packet of size 536
[ 4][t 1][1706169438.058619976][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 72
[ 4][t 1][1706169438.058773041][BufferedFd.h:216][!outconn]	Flush write: +140B[left:0B]
[ 4][t 1][1706169438.155931950][BufferedFd.h:207][!outconn]	Flush read: +540B[total:540B]
[ 4][t 1][1706169438.156007051][adnl-ext-connection.cpp:80][!outconn]	len=536
[ 4][t 1][1706169438.156101942][adnl-ext-connection.cpp:163][!outconn]	received packet of size 536
[ 4][t 1][1706169438.157544136][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 72
[ 

In [16]:
prev_block_data = [
    [i.to_data() for i in old_mc_blocks],
    key_block.to_data()
]

In [17]:
# Here we need to get all public libraries (todo: rewrite on new LS API if it'll be published)
# Alternative: deep_library_search 
def get_public_libs(block_hash):
    query = '''query{blocks(root_hash: "%s"){libs_hash}}''' % block_hash

    response = requests.post("https://dton.io/graphql", json={'query': query})
    
    data = response.json()['data']['blocks'][0]
    return data['libs_hash']

In [18]:
public_libs = get_public_libs(master.root_hash)

print("Found: ", len(public_libs), " public libs")

libs_data = lc.get_libraries(public_libs)

Found:  35  public libs


[ 4][t 1][1706169443.198479891][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 568
[ 4][t 1][1706169443.198819160][BufferedFd.h:216][!outconn]	Flush write: +636B[left:0B]
[ 4][t 1][1706169443.299615145][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:1288B]
[ 4][t 1][1706169443.299657106][adnl-ext-connection.cpp:80][!outconn]	len=29104
[ 4][t 1][1706169443.299929142][BufferedFd.h:207][!outconn]	Flush read: +3864B[total:5148B]
[ 4][t 1][1706169443.300276041][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:6436B]
[ 4][t 1][1706169443.308163166][BufferedFd.h:207][!outconn]	Flush read: +1288B[total:7724B]
[ 4][t 1][1706169443.308721066][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:10KB]
[ 4][t 1][1706169443.309861898][BufferedFd.h:207][!outconn]	Flush read: +2576B[total:12KB]
[ 4][t 1][1706169443.398398161][BufferedFd.h:207][!outconn]	Flush read: +15KB[total:28KB]
[ 4][t 1][1706169443.399066925][adnl-ext-connection.cpp:163][!outconn]	received packet of size 

In [19]:
# Now create emulator with all needed info
em = Emulator(config)

em.set_rand_seed(rand_seed)
em.set_prev_blocks_info(prev_block_data)
em.set_libs(libs_data)

[ 4][t 0][1706169443.635305166][mc-config.cpp:260]	needSpecialSmc flag set
[ 4][t 0][1706169443.635967016][mc-config.cpp:266]	smc dictionary created
[ 4][t 0][1706169443.636005878][mc-config.cpp:469]	workchain description dictionary created
[ 4][t 0][1706169443.638359070][mc-config.cpp:2061]	unpacked info for workchain 0: basic=true, active=true, accept_msgs=true, min_split=0, max_split=4


In [20]:
account_state.get_hash()

'A7433BA352CB4B0FD89CA06D7F923FB99EDE07932EDA64D4DAB8B650597E90DF'

In [21]:
def get_diff(tx1, tx2):
    tx1_tlb = Transaction()
    tx1_tlb = tx1_tlb.cell_unpack(tx1, True).dump()

    tx2_tlb = Transaction()
    tx2_tlb = tx2_tlb.cell_unpack(tx2, True).dump()

    diff = DeepDiff(tx1_tlb, tx2_tlb).to_dict()

    address = tx1_tlb['account_addr']
    del tx1_tlb
    del tx2_tlb

    return diff, address


In [22]:
initial_account_state = account_state

In [23]:
account_state = initial_account_state

In [24]:
for tx in whitelist_txs:
    # Get current transaction LT & now
    current_tx = tx.transaction

    tx_tlb = Transaction()
    tx_tlb = tx_tlb.cell_unpack(current_tx, True)

    lt = tx_tlb.lt
    now = tx_tlb.now
    
    # Get current trasaction in_msg
    current_tx_cs = current_tx.begin_parse()
    tmp = current_tx_cs.load_ref(as_cs=True)

    if tmp.load_bool():
        in_msg = tmp.load_ref()
    else:
        # TICK TOCK
        in_msg = None

    if in_msg is None:
        success = em.emulate_tick_tock_transaction(
                    account_state,
                    tx_tlb.description.is_tock,
                    now,
                    lt
                )
    else:
        # Emulate
        success = em.emulate_transaction(
            account_state,
            in_msg,
            now,
            lt
        )
        
    assert success
    
    # Emulation transaction equal current transaction
    if em.transaction.get_hash() != current_tx.get_hash():
        print(get_diff(current_tx, em.transaction.to_cell()))
        raise ValueError
    else:
        print(f"\tEmulate {current_tx.get_hash()} success\n\n")
    
    # Update account state, go to next transaction
    account_state = em.account.to_cell()
    print(account_state.dump_as_tlb("ShardAccount"))

[ 4][t 0][1706169446.268437147][transaction.cpp:444]	unpacking account 013721952970BF55969E523C8632C3088E9F45C87DA03A96346093CF481CBA91
[ 4][t 0][1706169446.268767118][transaction.cpp:275]	last_paid=1705492963; cells=1 bits=79 public_cells=0
[ 4][t 0][1706169446.269189119][transaction.cpp:551]	end of Account.unpack() for 0:013721952970BF55969E523C8632C3088E9F45C87DA03A96346093CF481CBA91 (balance = 1ng ; last_trans_lt = 43982925000007..43982925000008)
[ 4][t 0][1706169446.278834105][transaction.cpp:942]	Get storage_fees: 1705600207 due_payment: 0
[ 4][t 0][1706169446.278867006][transaction.cpp:739]	Last paid: 1705492963
[ 4][t 0][1706169446.278874874][transaction.cpp:944]	To pay: 948
[ 4][t 0][1706169446.278880119][transaction.cpp:952]	Last paid: 1705600207
[ 4][t 0][1706169446.278884172][transaction.cpp:968]	I'm here 4
[ 4][t 0][1706169446.278887033][transaction.cpp:1003]	I'm here 5
[ 4][t 0][1706169446.278947115][transaction.cpp:2883]	account is uninitialized and has zero balance, del

	Emulate EFD524E05D9EB175365596271D0C22B0A07A016FFCE01B366A1D51EDE75ED658 success


(account_descr
  account:account_none last_trans_hash:xEFD524E05D9EB175365596271D0C22B0A07A016FFCE01B366A1D51EDE75ED658 last_trans_lt:44012780000007)

({'dictionary_item_removed': [root['description']['storage_ph']['value']['storage_fees_due']['value']], 'values_changed': {"root['end_status']": {'new_value': 0, 'old_value': 3}, "root['total_fees']['grams']['amount']['len']": {'new_value': 0, 'old_value': 1}, "root['total_fees']['grams']['amount']['value']": {'new_value': 0, 'old_value': 1}, "root['state_update']['new_hash']": {'new_value': '4C552ACCDED7CB781DBA43143C216048C7087DF17948C30E5E3F2673585BC310', 'old_value': '90AEC8965AFABB16EBC3CB9B408EBAE71B618D78788BC80D09843593CAC98DA4'}, "root['description']['storage_ph']['value']['storage_fees_collected']['amount']['len']": {'new_value': 0, 'old_value': 1}, "root['description']['storage_ph']['value']['storage_fees_collected']['amount']['value']": {'new_

ValueError: 

[ 4][t 1][1706169453.614319324][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169453.614509106][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169453.689805269][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169453.689847231][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169453.689872026][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169463.691529989][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169463.691823244][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169463.760165215][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169463.760225058][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169463.760264158][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169473.761360884][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706169634.945242882][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169634.945702076][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169635.016128063][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169635.016180038][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169635.016197920][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169645.017916918][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169645.018518209][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169645.083153009][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169645.083292007][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169645.083310127][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169655.084879875][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706169816.579282999][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169816.580185175][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169816.647294998][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169816.647345066][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169816.647362232][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169826.648584127][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169826.649239063][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169826.745851994][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169826.745980978][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169826.746045113][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706169836.747952223][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706169997.960677147][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706169997.961075068][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706169998.024538279][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706169998.024605036][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706169998.024626970][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170008.025861979][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706170008.026257992][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706170008.087372065][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706170008.087426186][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706170008.087453127][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170018.088700056][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706170179.262531281][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706170179.263559103][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706170179.328255177][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706170179.328321218][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706170179.328341007][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170189.322191238][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706170189.322725296][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706170189.389950037][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706170189.390056133][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706170189.390139103][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170199.381698132][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17

[ 4][t 1][1706170360.524550915][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706170360.525243998][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706170360.604819059][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706170360.604901314][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706170360.605207205][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170370.606419086][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][1706170370.607107162][BufferedFd.h:216][!outconn]	Flush write: +80B[left:0B]
[ 4][t 1][1706170370.676372051][BufferedFd.h:207][!outconn]	Flush read: +80B[total:80B]
[ 4][t 1][1706170370.676469088][adnl-ext-connection.cpp:80][!outconn]	len=76
[ 4][t 1][1706170370.676515102][adnl-ext-connection.cpp:163][!outconn]	received packet of size 76
[ 4][t 1][1706170380.676985264][adnl-ext-connection.cpp:31][!outconn]	sending packet of size 12
[ 4][t 1][17